In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import re

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import TextRecognitionMode
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

import os
import sys
import time

from flask import Flask, render_template, request, redirect, url_for, Response
from jinja2 import Environment, FileSystemLoader

In [2]:
url = 'https://www.cnn.com/2019/10/11/business/costco-5-dollar-chicken/index.html'

In [3]:
def get_soup(url):
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    return soup

In [4]:
def get_logo(soup, url):
    slash = url[8:].find('/')
    if slash > -1:
        head = url[0:url[8:].find('/')+8]
    else:
        head = url

    ext = soup.find("link", rel="Shortcut Icon") or soup.find("link", rel="shortcut icon")
    if not ext:
         return ''
    else:
        ext = ext['href']
        if ext.find('http') == -1 and ext.find('.com') == -1:
            icon = head+ext
        else:
            icon = ext
        if urlparse(icon).netloc:
            return icon
        else:
            return ''

In [5]:
def is_cnn(url):
    return url.find('cnn.com') > -1

In [6]:
def get_body(soup, is_cnn):
    body_tags = ['a','div','p','h1','h2','h3','h4']
    body_ignore = ['Read More']
    
    text = soup.find_all(text=True)

    title = ''
    body = ''
    isbody = False
    
    if is_cnn:
        for t in text:
            tag = t.parent.name
            if tag == 'title':
                title = t
            elif tag == 'cite':
                isbody = True
            elif tag == 'body':
                break
            if isbody and tag in body_tags:
                if t not in body_ignore and t.find('http') == -1:
                    if prev == 'div':
                        body+='\n\n'
                    body += '{}'.format(t)
            prev = tag
    if not is_cnn or not body:
        for t in text:
            tag = t.parent.name
            if tag == 'title':
                title = t
            if tag in body_tags:
                t = t.replace('\n','')
                if len(t)>3 and t.find('http') == -1:
                    body += '{}'.format(t) + ' '
                    if len(t) > 25:
                        body += '\n\n'
    return title, body

In [7]:
def get_image(soup, is_cnn):
    images = soup.findAll('img')
    exists = []
    image_urls = []
    for image in images:
        image_url = image['src']

        if urlparse(image_url).netloc:
            if (image_url.find('.png') != -1 or 
               image_url.find('.jpg') != -1 or 
               image_url.find('.gif') != -1 or 
               image_url.find('.svg') != -1):
                if image_url[0:2] == '//':
                    image_url = image_url[2:]
                if image_url[0:4] != 'http':
                    image_url = 'http://' + image_url
                if is_cnn:
                    key = re.findall(r'/[0-9]*-',image_url)
                    if key and key[0][1:-1] not in exists:
                        exists.append(key[0][1:-1])
                        image_urls.append(image_url)
                else:
                    image_urls.append(image_url)

    return list(set(image_urls))

In [8]:
def generate_captions(images,cv_client):
    captions = []
    for image in images:
        try:
            description_results = cv_client.describe_image(image)
            if description_results.captions:
                captions.append(description_results.captions[0].text)
            else:
                captions.append('No caption available')
        except:
            captions.append('No caption available')
    return captions

In [ ]:
def simplify_text(text, summarizer):
    sentences = len(text)//150
    
    parser = PlaintextParser.from_string(text,Tokenizer("english"))
    summary = summarizer(parser.document, sentences)
    return ' '.join([str(x) for x in summary])

In [ ]:
def build_page(url,cv_client,summarizer):
    soup = get_soup(url)
    iscnn = is_cnn(url)
    
    title,unsimp_text = get_body(soup,iscnn)
    text = simplify_text(unsimp_text,summarizer)
    
    icon = get_logo(soup,url)
    images = get_image(soup,iscnn)
    captions = generate_captions(images,cv_client)

    return icon,title,text,images,captions

In [ ]:
def get_cv_client():
    endpoint = 'https://westcentralus.api.cognitive.microsoft.com'
    subscription_key = 'd0fa9162e03a4c7b9cf531e88956b3e3'
    cv_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
    return cv_client

In [ ]:
app = Flask(__name__)

@app.route('/')
def main():
    return render_template('landing.html')

@app.route('/serve_article', methods=['POST'])
def serve_article():
    try:
        cv_client, summarizer
    except NameError:
        summarizer = LexRankSummarizer()
        cv_client = get_cv_client()
    
    url = request.form['URL']
    icon,title,text,images,captions=build_page(url,cv_client,summarizer)
    print(images[0])
    print(text)
    return render_template('index.html',icon=icon,title=title,text=text,images=images,captions=captions)

if __name__ == "__main__":
    app.jinja_env.cache = {}
    app.run(host='127.0.0.1',port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Oct/2019 08:55:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2019 08:55:35] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2019 08:55:35] "GET /static/feature.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2019 08:55:35] "GET /static/carousel.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2019 08:56:46] "POST /serve_article HTTP/1.1" 200 -


http://cdn.cnn.com/cnnnext/dam/assets/190626114212-arbys-fake-carrots-meat-carrots-large-169.jpg
Markets Tech Media Success Perspectives Videos U.S. International Arabic Español Markets Tech Media Success Perspectives Videos Search U.S. International Arabic Español Markets Premarkets Dow 30 After-Hours Market Movers Fear & Greed World Markets Sector Performance Investing Markets Now Before the Bell Leading Indicator Global Energy Challenge Economy Energy Tech Innovate Gadget Mission: Ahead Upstarts Business Evolved Work Transformed Innovative Cities Unhackable Media Reliable Sources Success Boss Files Fresh Money Invest Ahead Cars Homes Wealth Coach Biz + Leisure Center Piece Perspectives Videos International Switzerland India Davos Reliable Sources Passion to Portfolio On: Germany Accessibility & CC About Us Newsletters World Politics Business Opinion Health Entertainment Tech Style Travel Sports Videos Coupons More JUST WATCHED Replay More Videos ... MUST WATCH  (16 Videos) It's only

127.0.0.1 - - [13/Oct/2019 08:57:45] "POST /serve_article HTTP/1.1" 200 -


http://cdn.cnn.com/cnnnext/dam/assets/190626114212-arbys-fake-carrots-meat-carrots-large-169.jpg
Markets Tech Media Success Perspectives Videos U.S. International Arabic Español Markets Tech Media Success Perspectives Videos Search U.S. International Arabic Español Markets Premarkets Dow 30 After-Hours Market Movers Fear & Greed World Markets Sector Performance Investing Markets Now Before the Bell Leading Indicator Global Energy Challenge Economy Energy Tech Innovate Gadget Mission: Ahead Upstarts Business Evolved Work Transformed Innovative Cities Unhackable Media Reliable Sources Success Boss Files Fresh Money Invest Ahead Cars Homes Wealth Coach Biz + Leisure Center Piece Perspectives Videos International Switzerland India Davos Reliable Sources Passion to Portfolio On: Germany Accessibility & CC About Us Newsletters World Politics Business Opinion Health Entertainment Tech Style Travel Sports Videos Coupons More JUST WATCHED Replay More Videos ... MUST WATCH  (16 Videos) It's only

...

http://cdn.cnn.com/cnnnext/dam/assets/190626114212-arbys-fake-carrots-meat-carrots-large-169.jpg
Markets Tech Media Success Perspectives Videos U.S. International Arabic Español Markets Tech Media Success Perspectives Videos Search U.S. International Arabic Español Markets Premarkets Dow 30 After-Hours Market Movers Fear & Greed World Markets Sector Performance Investing Markets Now Before the Bell Leading Indicator Global Energy Challenge Economy Energy Tech Innovate Gadget Mission: Ahead Upstarts Business Evolved Work Transformed Innovative Cities Unhackable Media Reliable Sources Success Boss Files Fresh Money Invest Ahead Cars Homes Wealth Coach Biz + Leisure Center Piece Perspectives Videos International Switzerland India Davos Reliable Sources Passion to Portfolio On: Germany Accessibility & CC About Us Newsletters World Politics Business Opinion Health Entertainment Tech Style Travel Sports Videos Coupons More JUST WATCHED Replay More Videos ... MUST WATCH  (16 Videos) It's only

127.0.0.1 - - [13/Oct/2019 09:05:17] "POST /serve_article HTTP/1.1" 200 -


http://cdn.cnn.com/cnnnext/dam/assets/191011183058-isis-families-escape-large-169.jpg
For more than a year, Joseph Roh illegally manufactured AR-15-style rifles in a warehouse south of Los Angeles.His customers, more than two dozen of whom were legally prohibited from possessing a firearm, could push a button, pull a lever, and walk away a short time later with a fully assembled, untraceable semi-automatic weapon for about $1,000, according to court records. Roh continued his black-market operation despite being warned in person by agents from the Bureau of Alcohol, Tobacco, Firearms and Explosives that he was breaking the law. But five years after raiding his business and indicting him, federal authorities quietly cut a deal with Roh earlier this year and agreed to drop the charges. Why? The judge in the case had issued a tentative order that, in the eyes of prosecutors, threatened to upend the decades-old Gun Control Act and "seriously undermine the ATF's ability to trace and regulat

127.0.0.1 - - [13/Oct/2019 09:09:32] "POST /serve_article HTTP/1.1" 200 -


http://cdn.cnn.com/cnnnext/dam/assets/191011183058-isis-families-escape-large-169.jpg
For more than a year, Joseph Roh illegally manufactured AR-15-style rifles in a warehouse south of Los Angeles.His customers, more than two dozen of whom were legally prohibited from possessing a firearm, could push a button, pull a lever, and walk away a short time later with a fully assembled, untraceable semi-automatic weapon for about $1,000, according to court records. Roh continued his black-market operation despite being warned in person by agents from the Bureau of Alcohol, Tobacco, Firearms and Explosives that he was breaking the law. But five years after raiding his business and indicting him, federal authorities quietly cut a deal with Roh earlier this year and agreed to drop the charges. Why? The judge in the case had issued a tentative order that, in the eyes of prosecutors, threatened to upend the decades-old Gun Control Act and "seriously undermine the ATF's ability to trace and regulat